In [32]:
import os
from torch.utils.data import Dataset
import pandas as pd
import torchaudio
from IPython.display import Audio

In [10]:
ANNOTATIONS_PATH = 'data/urbansound8k/metadata/UrbanSound8K.csv'
AUDIO_PATH = 'data/urbansound8k/audio/'

In [36]:
class UrbanSoundDataset(Dataset):
    def __init__(self, annotations_path, audio_path):
        self.annotations = pd.read_csv(annotations_path)
        self.audio_path = audio_path
        
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx): #dataset[0] -> dataset.__getitem__[0] 
        audio_sample_path = self._get_audio_sample_path(idx)
        label = self._get_audio_sample_label(idx)
        signal, sampling_rate = torchaudio.load(audio_sample_path)
        return signal, label
    
    def _get_audio_sample_path(self, idx):
        slice_file_name = self.annotations.iloc[idx, 0]
        fold = 'fold{0}'.format(self.annotations.iloc[idx, 5])
        path = os.path.join(self.audio_path, fold, slice_file_name)
        return path
    
    def _get_audio_sample_label(self, idx):
        return self.annotations.iloc[idx, 6]

In [38]:
dataset = UrbanSoundDataset(ANNOTATIONS_PATH, AUDIO_PATH)

In [46]:
dataset.annotations[:3]

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing


In [42]:
dataset[0]

(tensor([[-4.5776e-03, -4.8828e-03, -4.6082e-03,  ..., -5.7983e-04,
          -4.2725e-04,  3.0518e-05],
         [-4.5166e-03, -4.7913e-03, -4.6082e-03,  ..., -7.3242e-04,
          -5.4932e-04, -3.0518e-05]]),
 3)